### Model 1: Minimizing total swimmers time 

In [229]:
#Load packages
using DataFrames, CSV, JuMP, Gurobi, LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays, Distributions
const GRB_ENV = Gurobi.Env(output_flag=0);

In [253]:
# Load data
swimmer_times = CSV.read("swimmer_event_times.csv", DataFrame)
swimmer_stats = CSV.read("swimmer_stats.csv", DataFrame)
individual_event_order = CSV.read("individual_event_order.csv", DataFrame)
relay_event_order = CSV.read("relay_event_order.csv", DataFrame)
opponent_times = CSV.read("opponent_times.csv", DataFrame)

Row,Opponent,Event,Opponent_Time
,String31,String31,Float64
1,Group 1,200 Medley Relay,102.62
2,Group 2,200 Medley Relay,104.83
3,"Bastone, Alexandra",1000 Free,603.44
4,"Wood Prince, Piper",1000 Free,622.34
5,"Cruz-Abrams, Robin",1000 Free,653.79
6,"McDonald, Kaylee",200 Free,111.89
7,"Foster, Payton",200 Free,114.7
8,"Hakonardottir, Kristin",200 Free,116.2
9,"Hamlin, Molly",100 Back,54.94


In [304]:
# Create dictionaries
best_times = Dict((row[1], row[2]) => row[3] for row in eachrow(swimmer_times))
mean_times = Dict((row[1], row[2]) => row[3] for row in eachrow(swimmer_stats))
individual_event_order_list = individual_event_order[:, 2]
relay_event_positions = Dict(row[2] => row[3:6] for row in eachrow(relay_event_order))
swimmer_list = unique(swimmer_times[:, 1])
std_dev_times = Dict((row[1], row[2]) => row[4] for row in eachrow(swimmer_stats))


Dict{Tuple{String15, String31}, Float64} with 243 entries:
  ("50 Free", "Simons, Kailey")    => 0.713631
  ("50 Free", "Griffin, Miri")     => 0.226274
  ("200 Free", "Levy, Lauren")     => 0.0
  ("200 Free", "Smith, Sydney")    => 1.53237
  ("50 Breast", "Lu, Sarah")       => 0.0
  ("50 Free", "Smith, Sydney")     => 0.548698
  ("200 IM", "Yang, Iris")         => 0.0
  ("50 Free", "Levy, Lauren")      => 0.629265
  ("500 Free", "Bernard, Sarah")   => 0.0
  ("100 Fly", "Turvey, Alexandra") => 1.14551
  ("100 Free", "Zhu, Emma")        => 2.24153
  ("100 Free", "Kudela, Katie")    => 0.0
  ("400 IM", "Bernard, Sarah")     => 0.0
  ("200 Free", "Poag, Lillian")    => 0.0
  ("200 Breast", "Zhao, Kathy")    => 4.9474
  ("100 Fly", "Feliz, Mary")       => 0.0
  ("50 Breast", "Kudela, Katie")   => 0.609529
  ("50 Free", "Shipps, Stella")    => 0.513211
  ("100 Breast", "Lu, Sarah")      => 1.22329
  ("400 IM", "Crane, Jessie")      => 0.0
  ("200 Free", "Bernard, Sarah")   => 0.0
  ("100 Fl

In [255]:
# Parameters
swimmer_count = 32
num_individual_events = 12
num_relay_events = 2 # events 1 and 14
num_relay_positions = 4 # number of positions in each relay
num_groups = 3
num_places = 6 # 6 different places to finish a race

penalty_time = 99999  # A large penalty value to discourage invalid assignments

99999

In [6]:
model_1 = Model(() -> Gurobi.Optimizer(GRB_ENV))
#variables 
@variable(model_1, x[1:swimmer_count, 1:num_individual_events], Bin) #decision variable of if swimmer competes in event 
@variable(model_1, y[1:swimmer_count, 1:num_relay_positions, 1:num_relay_events, 1:num_groups], Bin) #decicsion variable if swimmer is 
#in the position for relay event in a certain group 

@objective(model_1, Min, 
    # Individual events
    sum(get(best_times, (swimmer_list[s], individual_event_order_list[e]), penalty_time) * x[s, e]
        for s in 1:swimmer_count, e in 1:num_individual_events) +
    
    # Relay events
    sum(get(best_times, (swimmer_list[s], relay_event_positions[relay_event_order[r, 2]][i]), penalty_time) * y[s, i, r, g]
        for s in 1:swimmer_count, i in 1:num_relay_positions, r in 1:num_relay_events, g in 1:num_groups))

# Each swimmer swims 3 individual events
@constraint(model_1, [s in 1:swimmer_count], 
    sum(x[s, e] for e in 1:num_individual_events) <= 3)

# Each individual event we have only one swimmer
@constraint(model_1, [e in 1:num_individual_events], 
    sum(x[s, e] for s in 1:swimmer_count) >= 1) 

# 4 persons per relay group
@constraint(model_1, [r in 1:num_relay_events, g in 1:num_groups], 
    sum(y[s, i, r, g] for s in 1:swimmer_count, i in 1:num_relay_positions) == 4)

# Can't swim back-to-back events
for s in 1:swimmer_count
    for e in 1:num_individual_events-1
        @constraint(model_1, x[s, e] + x[s, e+1] <= 1)
    end
end

# Each swimmer can only swim once in first relay
@constraint(model_1, [s in 1:swimmer_count], 
    sum(y[s, i, 1, g] for i in 1:num_relay_positions, g in 1:num_groups) <= 1)

# Each swimmer can only swim once in last relay
@constraint(model_1, [s in 1:swimmer_count], 
    sum(y[s, i, 2, g] for i in 1:num_relay_positions, g in 1:num_groups) <= 1)

# If swim in first relay, can't swim first individual event
@constraint(model_1, [s in 1:swimmer_count], 
    sum(y[s, i, 1, g] for i in 1:num_relay_positions, g in 1:num_groups) + x[s, 1] <= 1)

# If swim in last relay, can't swim last individual event
@constraint(model_1, [s in 1:swimmer_count], 
    sum(y[s, i, 2, g] for i in 1:num_relay_positions, g in 1:num_groups) + x[s, num_individual_events] <= 1)

# Each swimmer can only swim in one position in a group
@constraint(model_1, [r in 1:num_relay_events, g in 1:num_groups, i in 1:num_relay_positions], 
    sum(y[s, i, r, g] for s in 1:swimmer_count) == 1)

#optimizing 
optimize!(model_1)

if termination_status(model_1) == MOI.OPTIMAL
    println("Optimal solution found")
else
    println("No optimal solution found")
end

println("Total swimming time: ", objective_value(model_1))

Optimal solution found
Total swimming time: 2412.7999999999997


In [7]:
#printing out what events the swimmer are swimming and what times 
for s in 1:swimmer_count
    for e in 1:num_individual_events
        if value(x[s, e]) > 0.5
            swimmer = swimmer_list[s]
            event = individual_event_order[e, :event]
            best_time = get(best_times, (swimmer, event), "N/A")
            println("Swimmer $swimmer assigned to event $event (Best time: $best_time)")
        end
    end
end

Swimmer Augustyn, Kate assigned to event 200 Back (Best time: 123.96)
Swimmer Bernard, Sarah assigned to event 200 Breast (Best time: 136.67)
Swimmer Naveen, Annika assigned to event 50 Free (Best time: 23.53)
Swimmer Roberson, Ella assigned to event 1000 Free (Best time: 624.84)
Swimmer Roberson, Ella assigned to event 500 Free (Best time: 303.66)
Swimmer Roberson, Ella assigned to event 200 IM (Best time: 125.25)
Swimmer Simons, Kailey assigned to event 100 Breast (Best time: 60.29)
Swimmer Smith, Sydney assigned to event 200 Free (Best time: 116.24)
Swimmer Smith, Sydney assigned to event 100 Free (Best time: 52.48)
Swimmer Smith, Sydney assigned to event 100 Fly (Best time: 57.21)
Swimmer Swartwood, Belise assigned to event 200 Fly (Best time: 125.71)
Swimmer Yang, Iris assigned to event 100 Back (Best time: 55.8)


In [8]:
#prints out each relay group, the position (SHOULD BE 1-4), the swimmer, and the time 
for r in 1:num_relay_events
    for g in 1:num_groups
        println("Relay $(r), Group $(g):")
        for i in 1:num_relay_positions
            for s in 1:swimmer_count
                if value(y[s, i, r, g]) > 0.5
                    swimmer = swimmer_list[s]
                    event = relay_event_order[r, "swimmer_$(i)_event"]
                    best_time = get(best_times, (swimmer, event), "N/A")
                    println("  Position $(i): $(swimmer) (Best time: $(best_time))")
                end
            end
        end
    end
end

Relay 1, Group 1:
  Position 1: Smith, Sydney (Best time: 27.47)
  Position 2: Simons, Kailey (Best time: 30.34)
  Position 3: Beggs, Christina (Best time: 20.09)
  Position 4: Chen, Olivia (Best time: 24.48)
Relay 1, Group 2:
  Position 1: Augustyn, Kate (Best time: 26.5)
  Position 2: Bernard, Sarah (Best time: 29.97)
  Position 3: Naveen, Annika (Best time: 24.89)
  Position 4: Turvey, Alexandra (Best time: 23.61)
Relay 1, Group 3:
  Position 1: Yang, Iris (Best time: 26.18)
  Position 2: Li, Anna (Best time: 30.55)
  Position 3: Zhao, Kathy (Best time: 25.44)
  Position 4: Poag, Lillian (Best time: 24.25)
Relay 2, Group 1:
  Position 1: Smith, Sydney (Best time: 24.85)
  Position 2: Chen, Olivia (Best time: 24.48)
  Position 3: Beggs, Christina (Best time: 24.12)
  Position 4: Naveen, Annika (Best time: 23.53)
Relay 2, Group 2:
  Position 1: Augustyn, Kate (Best time: 24.88)
  Position 2: Kudela, Katie (Best time: 24.87)
  Position 3: Simons, Kailey (Best time: 24.55)
  Position 4:

In [9]:
#prints out the number of events swimmers are in, both individual and relay
for s in 1:swimmer_count
    individual_events = sum(value(x[s, e]) for e in 1:num_individual_events)
    relay_events = sum(value(y[s, i, r, g]) for i in 1:num_relay_positions, r in 1:num_relay_events, g in 1:num_groups)
    total_events = individual_events + relay_events
    println("Swimmer $(swimmer_list[s]): $(total_events) events ($(individual_events) individual, $(relay_events) relay)")
end

Swimmer Adler, Lauren: -0.0 events (-0.0 individual, -0.0 relay)
Swimmer Augustyn, Kate: 3.0 events (1.0 individual, 2.0 relay)
Swimmer Beggs, Christina: 2.0 events (-0.0 individual, 2.0 relay)
Swimmer Bernard, Sarah: 2.0 events (1.0 individual, 1.0 relay)
Swimmer Chen, Olivia: 2.0 events (-0.0 individual, 2.0 relay)
Swimmer Chun, Sydney: -0.0 events (-0.0 individual, -0.0 relay)
Swimmer Crane, Jessie: -0.0 events (-0.0 individual, -0.0 relay)
Swimmer Eppinger, Aria: -0.0 events (-0.0 individual, -0.0 relay)
Swimmer Feliz, Mary: 1.0 events (-0.0 individual, 1.0 relay)
Swimmer Griffin, Miri: -0.0 events (-0.0 individual, -0.0 relay)
Swimmer Kim, Jolie: -0.0 events (-0.0 individual, -0.0 relay)
Swimmer Kudela, Katie: 1.0 events (-0.0 individual, 1.0 relay)
Swimmer Levy, Lauren: -0.0 events (-0.0 individual, -0.0 relay)
Swimmer Li, Anna: 1.0 events (-0.0 individual, 1.0 relay)
Swimmer Lu, Sarah: -0.0 events (-0.0 individual, -0.0 relay)
Swimmer Naveen, Annika: 3.0 events (1.0 individual, 

### Model 2: Maximizing Total Points

In [256]:
# Points vectors
p_i = [9, 4, 3, 2, 1, 0]  # Points for individual events
p_r = [11, 4, 2, 0, 0, 0]  # Points for relay events

6-element Vector{Int64}:
 11
  4
  2
  0
  0
  0

In [345]:
num_places = 6 # 6 different places to finish a race
num_opponent = 21

21

In [259]:
opponent_list = unique(opponent_times[:, :Opponent])
opponent_time_dict = Dict((row[:Event], row[:Opponent]) => row[:Opponent_Time] for row in eachrow(opponent_times))

Dict{Tuple{String31, String31}, Float64} with 42 entries:
  ("50 Free", "Brenner, Mandy")           => 24.15
  ("200 Free Relay", "Group 4")           => 203.88
  ("200 Back", "Brock, Lizzie")           => 126.95
  ("100 Breast", "Augustyn, Gabi")        => 63.68
  ("100 Breast", "Denisenko, Aleksandra") => 65.21
  ("100 Back", "Wieclawek, Blythe")       => 56.6
  ("200 Free Relay", "Group 6")           => 99999.0
  ("100 Free", "Brenner, Mandy")          => 52.89
  ("1000 Free", "Cruz-Abrams, Robin")     => 653.79
  ("200 Fly", "Yoon, Grace")              => 125.89
  ("1000 Free", "Wood Prince, Piper")     => 622.34
  ("100 Back", "Hamlin, Molly")           => 54.94
  ("500 Free", "Bastone, Alexandra")      => 291.62
  ("200 IM", "Iannaccone, Stephanie")     => 123.86
  ("100 Back", "Wilhelm, Kiley")          => 58.88
  ("50 Free", "Alas, Isabella")           => 24.02
  ("200 Breast", "Iannaccone, Stephanie") => 136.48
  ("200 Free", "Hakonardottir, Kristin")  => 116.2
  ("50 Free", "

In [260]:
mean_times = Dict((row[1], row[2]) => row[3] for row in eachrow(swimmer_stats))

Dict{Tuple{String15, String31}, Float64} with 243 entries:
  ("50 Free", "Simons, Kailey")    => 24.06
  ("50 Free", "Griffin, Miri")     => 26.14
  ("200 Free", "Levy, Lauren")     => 122.47
  ("200 Free", "Smith, Sydney")    => 112.59
  ("50 Breast", "Lu, Sarah")       => 44.39
  ("50 Free", "Smith, Sydney")     => 23.95
  ("200 IM", "Yang, Iris")         => 134.89
  ("50 Free", "Levy, Lauren")      => 26.38
  ("500 Free", "Bernard, Sarah")   => 300.4
  ("100 Fly", "Turvey, Alexandra") => 57.4
  ("100 Free", "Zhu, Emma")        => 54.4
  ("100 Free", "Kudela, Katie")    => 55.77
  ("400 IM", "Bernard, Sarah")     => 268.62
  ("200 Free", "Poag, Lillian")    => 116.19
  ("200 Breast", "Zhao, Kathy")    => 152.82
  ("100 Fly", "Feliz, Mary")       => 59.62
  ("50 Breast", "Kudela, Katie")   => 30.56
  ("50 Free", "Shipps, Stella")    => 25.11
  ("100 Breast", "Lu, Sarah")      => 74.08
  ("400 IM", "Crane, Jessie")      => 277.87
  ("200 Free", "Bernard, Sarah")   => 116.07
  ("100 Fly

In [351]:
# Create model
M = 99999
#num_groups = 6

model_2 = Model(Gurobi.Optimizer)

# Variables
@variable(model_2, x[1:swimmer_count, 1:num_individual_events, 1:num_places], Bin)
#@variable(model_2, y[1:swimmer_count, 1:num_relay_positions, 1:num_relay_events, 1:num_groups], Bin)
@variable(model_2, z_iok[1:num_individual_events, 1:num_opponent, 1:num_places], Bin)
#@variable(model_2, z_gk[1:num_groups, 1:num_places], Bin)
#@variable(model_2, S_mk[1:num_groups, 1:num_places], Bin)
#@variable(model_2, group_time[1:num_groups, 1:num_relay_events])

# Objective function
@objective(model_2, Max, 
    sum(p_i[k] * x[s, i, k] for s in 1:swimmer_count, i in 1:num_individual_events, k in 1:num_places)
    #+ sum(p_r[k] * z_gk[g, k] for g in 1:num_groups, k in 1:num_places)
)

# Constraints
# Each swimmer swims max 3 individual events
@constraint(model_2, [s in 1:swimmer_count], 
    sum(x[s, i, k] for i in 1:num_individual_events, k in 1:num_places) <= 3)

#=
@constraint(model_2, [i in 1:num_individual_events, s in 1:swimmer_count], 
    sum(x[s, i, k] for k in 1:num_places) <= 1)

# 4 persons per relay group
@constraint(model_2, [r in 1:num_relay_events, g in 1:num_groups], 
    sum(y[s, i, r, g] for s in 1:swimmer_count, i in 1:num_relay_positions) == 4)
=#
# Can't swim back-to-back events
for s in 1:swimmer_count
    for i in 1:num_individual_events - 1
        @constraint(model_2, sum(x[s, i, k] + x[s, i + 1, k] for k in 1:num_places) <= 1)
    end
end

# Ensure each opponent swims the events they are scheduled for
@constraint(model_2, [i in 1:num_individual_events, o in 1:num_opponent],
    sum(z_iok[i, o, k] for k in 1:num_places) == 
    (haskey(opponent_time_dict, (individual_event_order_list[i], opponent_list[o])) ? 1 : 0))
#=
# Each swimmer can only swim once in first relay
@constraint(model_2, [s in 1:swimmer_count], 
    sum(y[s, i, 1, g] for i in 1:num_relay_positions, g in 1:num_groups) <= 1)

# Each swimmer can only swim once in last relay
@constraint(model_2, [s in 1:swimmer_count], 
    sum(y[s, i, 2, g] for i in 1:num_relay_positions, g in 1:num_groups) <= 1)

# Each swimmer can only swim in one position in a group
@constraint(model_2, [r in 1:num_relay_events, g in 1:num_groups, i in 1:num_relay_positions], 
    sum(y[s, i, r, g] for s in 1:swimmer_count) == 1)
=#
# Ensure that for each event, the time of the swimmer or opponent in place k1 is less than or equal to the time of the swimmer or opponent in place k2
@constraint(model_2, [i in 1:num_individual_events, k1 in 1:(num_places - 1), k2 in (k1+1):num_places],
    sum(x[s, i, k1] * (get(mean_times, (individual_event_order_list[i], swimmer_list[s]), M) < M ? get(mean_times, (individual_event_order_list[i],swimmer_list[s]), M) : M) for s in 1:swimmer_count) +
    sum(z_iok[i, o, k1] * (get(opponent_time_dict, (individual_event_order_list[i], opponent_list[o]), M) < M ? get(opponent_time_dict, (individual_event_order_list[i], opponent_list[o]), M) : M) for o in 1:num_opponent) <=
    sum(x[s, i, k2] * (get(mean_times, (individual_event_order_list[i], swimmer_list[s]), M) < M ? get(mean_times, (individual_event_order_list[i], swimmer_list[s]), M) : M) for s in 1:swimmer_count) +
    sum(z_iok[i, o, k2] * (get(opponent_time_dict, (individual_event_order_list[i], opponent_list[o]), M) < M ? get(opponent_time_dict, (individual_event_order_list[i], opponent_list[o]), M) : M) for o in 1:num_opponent)
)

# Only one swimmer/opponent per place per event
@constraint(model_2, [i in 1:num_individual_events, k in 1:num_places],
    sum(x[s, i, k] for s in 1:swimmer_count) + sum(z_iok[i, o, k] for o in 1:num_opponent) == 1)
#=
@constraint(model_2, [i in 1:num_individual_events, k in 1:num_places],
    sum(x[s, i, k] for s in 1:swimmer_count) <= 1)

@constraint(model_2, [i in 1:num_individual_events, k in 1:num_places],
    sum(z_iok[i, o, k] for o in 1:num_opponent) <= 1)  

#set group time to be the sum of the times of the swimmers in that group
@constraint(model_2, [g in 1:num_groups, r in 1:num_relay_events],
    group_time[g, r] == sum(
        y[s, i, r, g] * get(mean_times, (swimmer_list[s], relay_event_positions[relay_event_order[r, 2]][i]), M) 
        for s in 1:swimmer_count, i in 1:num_relay_positions
    )
)

# Each swimmer can only swim in one position in a group
@constraint(model_2, [r in 1:num_relay_events, g in 1:num_groups, i in 1:num_relay_positions], 
    sum(y[s, i, r, g] for s in 1:swimmer_count) == 1)

# If swim in first relay, can't swim first individual event
@constraint(model_2, [s in 1:swimmer_count], 
    sum(y[s, i, 1, g] for i in 1:num_relay_positions, g in 1:num_groups) + sum(x[s, 1, k] for k in 1:num_places) <= 1)

# If swim in last relay, can't swim last individual event
@constraint(model_2, [s in 1:swimmer_count], 
    sum(y[s, i, 2, g] for i in 1:num_relay_positions, g in 1:num_groups) + sum(x[s, num_individual_events, k] for k in 1:num_places) <= 1)
=#
# Solve the model
optimize!(model_2)

if termination_status(model_2) == MOI.OPTIMAL
    println("Optimal solution found")
else
    println("No optimal solution found")
end

println("Total points won: ", objective_value(model_2))

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-21
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[x86] - Darwin 22.5.0 22F66)

CPU model: Intel(R) Core(TM) i7-8569U CPU @ 2.80GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 888 rows, 3816 columns and 30936 nonzeros
Model fingerprint: 0xadbde086
Variable types: 0 continuous, 3816 integer (3816 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [1e+00, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 356 rows and 2605 columns
Presolve time: 0.07s
Presolved: 532 rows, 1211 columns, 5701 nonzeros
Variable types: 0 continuous, 1211 integer (1167 binary)
Found heuristic solution: objective 48.0000000
Found heuristic solution: objective 69.0000000

Root relaxation: objective 7.699990e+01, 583 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node

In [352]:
println("\nIndividual Events Points:")
for s in 1:swimmer_count
    for i in 1:num_individual_events
        for k in 1:num_places
            if value(x[s, i, k]) > 0.5
                swimmer = swimmer_list[s]
                event = individual_event_order_list[i]
                place = k
                points = p_i[k]
                time = get(mean_times, (event, swimmer), penalty_time)  # Retrieve swimmer's time
                println("Swimmer: $swimmer, Event: $event, Place: $place, Points: $points, Time: $time")
            end
        end
    end
end



Individual Events Points:
Swimmer: Adler, Lauren, Event: 1000 Free, Place: 4, Points: 2, Time: 632.74
Swimmer: Adler, Lauren, Event: 100 Fly, Place: 6, Points: 0, Time: 99999
Swimmer: Augustyn, Kate, Event: 50 Free, Place: 4, Points: 2, Time: 24.13
Swimmer: Augustyn, Kate, Event: 200 Back, Place: 2, Points: 4, Time: 122.9
Swimmer: Augustyn, Kate, Event: 500 Free, Place: 5, Points: 1, Time: 305.38
Swimmer: Beggs, Christina, Event: 1000 Free, Place: 3, Points: 3, Time: 623.61
Swimmer: Beggs, Christina, Event: 100 Back, Place: 3, Points: 3, Time: 57.57
Swimmer: Bernard, Sarah, Event: 100 Breast, Place: 3, Points: 3, Time: 64.43
Swimmer: Bernard, Sarah, Event: 500 Free, Place: 2, Points: 4, Time: 300.4
Swimmer: Chun, Sydney, Event: 200 Fly, Place: 6, Points: 0, Time: 131.46
Swimmer: Crane, Jessie, Event: 1000 Free, Place: 5, Points: 1, Time: 637.64
Swimmer: Eppinger, Aria, Event: 200 Breast, Place: 6, Points: 0, Time: 153.63
Swimmer: Feliz, Mary, Event: 200 Fly, Place: 5, Points: 1, Time:

In [353]:
println("\nRelay Events Points and Assignments:")
for r in 1:num_relay_events
    for g in 1:num_groups
        # Check if this group got points
        for k in 1:num_places
            if value(z_gk[g, k]) > 0.5
                points = p_r[k]
                println("Relay: $r Group: $g Place: $k Points: $points")
            end
        end
        
        # Print relay assignments
        for k in 1:num_places
            if value(z_gk[g, k]) > 0.5
                println("Relay Group $g, Place: $k, Points: $(p_r[k])")
                for i in 1:num_relay_positions
                    for s in 1:swimmer_count
                        if value(y[s, i, r, g]) > 0.5
                            swimmer = swimmer_list[s]
                            event = relay_event_order[r, :event]
                            time = get(mean_times, (event, swimmer), penalty_time)
                            println("  Swimmer: $swimmer, Position: $i, Event: $event, Best time: $time")
                        end
                    end
                end
            end
        end
    end
end


Relay Events Points and Assignments:


In [354]:
# Calculate points for opponent in individual events
opponent_individual_points = 0
for i in 1:num_individual_events
    for k in 1:num_places
        for o in 1:num_opponent
        if value(z_iok[i,o,k]) > 0.5  # Check if the opponent is assigned to place k in event i
            opponent_individual_points += p_i[k]
            end
        end
    end
end

# Calculate points for opponent in relay events
opponent_relay_points = 0
for m in 1:3  # Iterate over opponent relay groups
    for k in 1:num_places
        if value(S_mk[m,k]) > 0.5  # Check if the opponent relay group m is assigned to place k
            opponent_relay_points += p_r[k]
        end
    end
end

# Total opponent points
total_opponent_points = opponent_individual_points + opponent_relay_points

# Print results
println("Opponent points from individual events: ", opponent_individual_points)
println("Opponent points from relay events: ", opponent_relay_points)
println("Total opponent points: ", total_opponent_points)

Opponent points from individual events: 154
Opponent points from relay events: 0
Total opponent points: 154


In [355]:
println("\nOpponent Assignments:")
for i in 1:num_individual_events
    for o in 1:num_opponent
        for k in 1:num_places
            if value(z_iok[i, o, k]) > 0.5
                opponent = opponent_list[o]
                event = individual_event_order_list[i]
                place = k
                time = get(opponent_time_dict, (event, opponent), "N/A")
                println("Opponent: $opponent, Event: $event, Place: $place, Time: $time")
            end
        end
    end
end



Opponent Assignments:
Opponent: Bastone, Alexandra, Event: 1000 Free, Place: 1, Time: 603.44
Opponent: Wood Prince, Piper, Event: 1000 Free, Place: 2, Time: 622.34
Opponent: Cruz-Abrams, Robin, Event: 1000 Free, Place: 6, Time: 653.79
Opponent: McDonald, Kaylee, Event: 200 Free, Place: 1, Time: 111.89
Opponent: Foster, Payton, Event: 200 Free, Place: 5, Time: 114.7
Opponent: Hakonardottir, Kristin, Event: 200 Free, Place: 6, Time: 116.2
Opponent: Hamlin, Molly, Event: 100 Back, Place: 1, Time: 54.94
Opponent: Wieclawek, Blythe, Event: 100 Back, Place: 2, Time: 56.6
Opponent: Wilhelm, Kiley, Event: 100 Back, Place: 5, Time: 58.88
Opponent: Augustyn, Gabi, Event: 100 Breast, Place: 1, Time: 63.68
Opponent: Iannaccone, Stephanie, Event: 100 Breast, Place: 2, Time: 63.75
Opponent: Denisenko, Aleksandra, Event: 100 Breast, Place: 4, Time: 65.21
Opponent: Padilla, Kenzie, Event: 200 Fly, Place: 1, Time: 121.14
Opponent: Lu, Sydney, Event: 200 Fly, Place: 2, Time: 122.45
Opponent: Yoon, Grac

In [356]:
# Loop over each event
for i in 1:num_individual_events
    println("Results for Event $i:")

    # Loop over each position (1st, 2nd, etc.)
    for k in 1:num_places
        # Find the swimmer or opponent in position k
        swimmer_in_place = nothing
        opponent_in_place = nothing
        time_in_place = nothing

        # Check if a swimmer is in place k
        for s in 1:swimmer_count
            if value(x[s, i, k]) > 0.5  # x[s, i, k] indicates the swimmer in place k
                swimmer_in_place = swimmer_list[s]
                time_in_place = get(mean_times, (individual_event_order_list[i], swimmer_list[s]), penalty_time)
                break
            end
        end

        # Check if an opponent is in place k
        if isnothing(swimmer_in_place)  # Only check if no swimmer was found in place k
            for o in 1:num_opponent
                if value(z_iok[i, o, k]) > 0.5  # z_iok[i, o, k] indicates the opponent in place k
                    opponent_in_place = opponent_list[o]
                    time_in_place = get(opponent_time_dict, (individual_event_order_list[i], opponent_list[o]), penalty_time)
                    break
                end
            end
        end

        # Print the result for place k
        if !isnothing(swimmer_in_place)
            println("Place $k: Swimmer $swimmer_in_place, Time: $time_in_place")
        elseif !isnothing(opponent_in_place)
            println("Place $k: Opponent $opponent_in_place, Time: $time_in_place")
        else
            println("Place $k: No participant")
        end
    end
end


Results for Event 1:
Place 1: Opponent Bastone, Alexandra, Time: 603.44
Place 2: Opponent Wood Prince, Piper, Time: 622.34
Place 3: Swimmer Beggs, Christina, Time: 623.61
Place 4: Swimmer Adler, Lauren, Time: 632.74
Place 5: Swimmer Crane, Jessie, Time: 637.64
Place 6: Opponent Cruz-Abrams, Robin, Time: 653.79
Results for Event 2:
Place 1: Opponent McDonald, Kaylee, Time: 111.89
Place 2: Swimmer Roberson, Ella, Time: 112.1
Place 3: Swimmer Smith, Sydney, Time: 112.59
Place 4: Swimmer Swartwood, Belise, Time: 113.86
Place 5: Opponent Foster, Payton, Time: 114.7
Place 6: Opponent Hakonardottir, Kristin, Time: 116.2
Results for Event 3:
Place 1: Opponent Hamlin, Molly, Time: 54.94
Place 2: Opponent Wieclawek, Blythe, Time: 56.6
Place 3: Swimmer Beggs, Christina, Time: 57.57
Place 4: Swimmer Yang, Iris, Time: 57.72
Place 5: Opponent Wilhelm, Kiley, Time: 58.88
Place 6: Swimmer Xu, Chloe, Time: 99999
Results for Event 4:
Place 1: Opponent Augustyn, Gabi, Time: 63.68
Place 2: Opponent Iannac

In [357]:
# Create model
M = 99999
#num_groups = 6

model_3 = Model(Gurobi.Optimizer)

# Variables
@variable(model_3, x[1:swimmer_count, 1:num_individual_events, 1:num_places], Bin)
@variable(model_3, z_iok[1:num_individual_events, 1:num_opponent, 1:num_places], Bin)
@variable(model_3, uncertain_time[1:swimmer_count, 1:num_individual_events])

# Objective function
@objective(model_3, Max, 
    sum(p_i[k] * x[s, i, k] for s in 1:swimmer_count, i in 1:num_individual_events, k in 1:num_places)
)

# Loop to conditionally add the constraints
for s in 1:swimmer_count
    for i in 1:num_individual_events
        if haskey(mean_times, (individual_event_order_list[i], swimmer_list[s])) && 
           haskey(std_dev_times, (individual_event_order_list[i], swimmer_list[s]))
            
            # Get the mean time and standard deviation for this swimmer and event
            mean_time = get(mean_times, (individual_event_order_list[i], swimmer_list[s]), M)
            std_dev = get(std_dev_times, (individual_event_order_list[i], swimmer_list[s]), M)
            
            # Create the lower and upper bound constraints
            @constraint(model_3, uncertain_time[s, i] >= mean_time - std_dev)
            @constraint(model_3, uncertain_time[s, i] <= mean_time + std_dev)
            
        else
            # If the mean time or standard deviation is not found, you can set a default value or skip
            @constraint(model_3, uncertain_time[s, i] == M)  # Or another default value like NaN
        end
    end
end

# Constraints
# Each swimmer swims max 3 individual events
@constraint(model_3, [s in 1:swimmer_count], 
    sum(x[s, i, k] for i in 1:num_individual_events, k in 1:num_places) <= 3)

# Can't swim back-to-back events
for s in 1:swimmer_count
    for i in 1:num_individual_events - 1
        @constraint(model_3, sum(x[s, i, k] + x[s, i + 1, k] for k in 1:num_places) <= 1)
    end
end

#three swimmers per event
@constraint(model_3, [i in 1:num_individual_events], 
    sum(x[s, i, k] for s in 1:swimmer_count, k in 1:num_places) == 3)

# Ensure each opponent swims the events they are scheduled for
@constraint(model_3, [i in 1:num_individual_events, o in 1:num_opponent],
    sum(z_iok[i, o, k] for k in 1:num_places) == 
    (haskey(opponent_time_dict, (individual_event_order_list[i], opponent_list[o])) ? 1 : 0))

# Ensure that for each event, the time of the swimmer or opponent in place k1 is less than or equal to the time of the swimmer or opponent in place k2
@constraint(model_3, [i in 1:num_individual_events, k1 in 1:(num_places - 1), k2 in (k1+1):num_places],
    sum(x[s, i, k1] * uncertain_time[s,i] for s in 1:swimmer_count) +
    sum(z_iok[i, o, k1] * (get(opponent_time_dict, (individual_event_order_list[i], opponent_list[o]), M) < M ? get(opponent_time_dict, (individual_event_order_list[i], opponent_list[o]), M) : M) for o in 1:num_opponent) <=
    sum(x[s, i, k2] * uncertain_time[s,i] for s in 1:swimmer_count) +
    sum(z_iok[i, o, k2] * (get(opponent_time_dict, (individual_event_order_list[i], opponent_list[o]), M) < M ? get(opponent_time_dict, (individual_event_order_list[i], opponent_list[o]), M) : M) for o in 1:num_opponent)
)

# Only one swimmer/opponent per place per event
@constraint(model_3, [i in 1:num_individual_events, k in 1:num_places],
    sum(x[s, i, k] for s in 1:swimmer_count) + sum(z_iok[i, o, k] for o in 1:num_opponent) == 1)

# Solve the model
optimize!(model_3)

if termination_status(model_3) == MOI.OPTIMAL
    println("Optimal solution found")
else
    println("No optimal solution found")
end

println("Total points won: ", objective_value(model_3))

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-21
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[x86] - Darwin 22.5.0 22F66)

CPU model: Intel(R) Core(TM) i7-8569U CPU @ 2.80GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1303 rows, 4200 columns and 14743 nonzeros
Model fingerprint: 0x55f7879e
Model has 180 quadratic constraints
Variable types: 384 continuous, 3816 integer (3816 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [2e+01, 1e+05]
  Objective range  [1e+00, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+05]
Presolve removed 799 rows and 1557 columns
Presolve time: 0.03s
Presolved: 2529 rows, 3381 columns, 32163 nonzeros
Variable types: 861 continuous, 2520 integer (2520 binary)
Found heuristic solution: objective 57.0000000

Root relaxation: objective 1.318444e+02, 915 iterations

In [358]:
# Loop over each event
for i in 1:num_individual_events
    println("Results for Event $i:")

    # Loop over each position (1st, 2nd, etc.)
    for k in 1:num_places
        # Initialize placeholders for results
        swimmer_in_place = nothing
        opponent_in_place = nothing
        time_in_place = nothing

        # Check if a swimmer is in place k
        for s in 1:swimmer_count
            if value(x[s, i, k]) > 0.5  # x[s, i, k] indicates the swimmer in place k
                swimmer_in_place = swimmer_list[s]
                time_in_place = value(uncertain_time[s, i])  # Use the optimized uncertain time
                break
            end
        end

        # Check if an opponent is in place k
        if isnothing(swimmer_in_place)  # Only check if no swimmer was found in place k
            for o in 1:num_opponent
                if value(z_iok[i, o, k]) > 0.5  # z_iok[i, o, k] indicates the opponent in place k
                    opponent_in_place = opponent_list[o]
                    time_in_place = get(opponent_time_dict, (individual_event_order_list[i], opponent_list[o]), penalty_time)
                    break
                end
            end
        end

        # Print the result for place k
        if !isnothing(swimmer_in_place)
            println("Place $k: Swimmer $swimmer_in_place, Time: $time_in_place (Uncertain)")
        elseif !isnothing(opponent_in_place)
            println("Place $k: Opponent $opponent_in_place, Time: $time_in_place")
        else
            println("Place $k: No participant")
        end
    end
end


Results for Event 1:
Place 1: Opponent Bastone, Alexandra, Time: 603.44
Place 2: Swimmer Beggs, Christina, Time: 604.54640118 (Uncertain)
Place 3: Opponent Wood Prince, Piper, Time: 622.34
Place 4: Swimmer Adler, Lauren, Time: 632.74 (Uncertain)
Place 5: Swimmer Crane, Jessie, Time: 637.64 (Uncertain)
Place 6: Opponent Cruz-Abrams, Robin, Time: 653.79
Results for Event 2:
Place 1: Swimmer Roberson, Ella, Time: 110.98069466 (Uncertain)
Place 2: Swimmer Smith, Sydney, Time: 111.057627889 (Uncertain)
Place 3: Opponent McDonald, Kaylee, Time: 111.89
Place 4: Swimmer Swartwood, Belise, Time: 113.86 (Uncertain)
Place 5: Opponent Foster, Payton, Time: 114.7
Place 6: Opponent Hakonardottir, Kristin, Time: 116.2
Results for Event 3:
Place 1: Swimmer Chen, Olivia, Time: 54.578084784 (Uncertain)
Place 2: Opponent Hamlin, Molly, Time: 54.94
Place 3: Swimmer Beggs, Christina, Time: 55.76813985 (Uncertain)
Place 4: Swimmer Augustyn, Kate, Time: 56.6 (Uncertain)
Place 5: Opponent Wieclawek, Blythe, T

In [359]:
opponent_individual_points = 0
for i in 1:num_individual_events
    for k in 1:num_places
        for o in 1:num_opponent
        if value(z_iok[i,o,k]) > 0.5  # Check if the opponent is assigned to place k in event i
            opponent_individual_points += p_i[k]
            end
        end
    end
end
println("Opponent points from individual events: ", opponent_individual_points)

Opponent points from individual events: 111
